<a href="https://colab.research.google.com/github/Chandra2205/perkinson-disease_ML/blob/main/Final_BTP_ML_project.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install catboost
#CatBoost is a fast, scalable, high performance gradient boosting on decision trees library.
#Used for ranking, classification, regression and other ML tasks.

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [ ]:
#GENERAL
import pandas as pd
import numpy as np
import seaborn as sns     #data visualization library built on top of Matplotlib.
import matplotlib.pyplot as plt
#PATH PROCESS
import os
import os.path
from pathlib import Path
import glob
#IMAGE PROCESS
from PIL import Image
from keras.preprocessing import image#In Keras, load_img() function is used to load image
from tensorflow.keras.preprocessing.image import ImageDataGenerator
import cv2
#SCALER & TRANSFORMATION
from sklearn.preprocessing import StandardScaler#a feature by subtracting the mean and then scaling to unit variance
from sklearn.preprocessing import MinMaxScaler
from keras.utils.np_utils import to_categorical
from sklearn.model_selection import train_test_split
from keras import regularizers#Regularizers allow you to apply penalties on layer parameters or layer activity during optimization.
from sklearn.preprocessing import LabelEncoder # Encode target labels with value between 0 and n_classes-1. This transformer should be used to encode target values
#ACCURACY CONTROL
from sklearn.metrics import confusion_matrix, accuracy_score, classification_report, roc_auc_score, roc_curve
from sklearn.model_selection import GridSearchCV, cross_val_score
from sklearn.metrics import mean_squared_error, r2_score
#OPTIMIZER
from keras.optimizers import RMSprop,Adam,Optimizer,Optimizer
#MODEL LAYERS
from tensorflow.keras.models import Sequential
from keras.layers import Dense, Dropout, Flatten, Conv2D, MaxPool2D, BatchNormalization,MaxPooling2D,BatchNormalization,\
                        Permute, TimeDistributed, Bidirectional,GRU, SimpleRNN, LSTM, GlobalAveragePooling2D, SeparableConv2D
from keras import models
from keras import layers
import tensorflow as tf
from keras.applications import VGG16,VGG19,inception_v3
from keras import backend as K
from keras.utils import plot_model
#SKLEARN CLASSIFIER
from xgboost import XGBClassifier, XGBRegressor
from lightgbm import LGBMClassifier, LGBMRegressor
from catboost import CatBoostClassifier, CatBoostRegressor
from sklearn.linear_model import LogisticRegression
from sklearn.naive_bayes import GaussianNB
from sklearn.ensemble import RandomForestClassifier, RandomForestRegressor
from sklearn.ensemble import GradientBoostingClassifier, GradientBoostingRegressor
from sklearn.ensemble import BaggingRegressor
from sklearn.tree import DecisionTreeClassifier, DecisionTreeRegressor
from sklearn.neural_network import MLPClassifier, MLPRegressor
from sklearn.neighbors import KNeighborsClassifier, KNeighborsRegressor
from sklearn.linear_model import LinearRegression
from sklearn.cross_decomposition import PLSRegression
from sklearn.linear_model import Ridge
from sklearn.linear_model import RidgeCV
from sklearn.linear_model import Lasso
from sklearn.linear_model import LassoCV
from sklearn.linear_model import ElasticNet
from sklearn.linear_model import ElasticNetCV

#IGNORING WARNINGS
from warnings import filterwarnings
filterwarnings("ignore",category=DeprecationWarning)
filterwarnings("ignore", category=FutureWarning)
filterwarnings("ignore", category=UserWarning)

In [ ]:
!unzip "/content/BTP.zip"

Archive:  /content/BTP.zip
   creating: BTP/
  inflating: BTP/ICACT48636.2020.9061497.pdf  
  inflating: BTP/202004021910156883ajay_misra_geo_Digital_Image_Enhancement.pdf  
  inflating: BTP/Image Enhancement in Spatial Domain.pdf  
  inflating: BTP/Article18.pdf       
  inflating: BTP/1003.4053.pdf       
   creating: BTP/dataset/
  inflating: BTP/dataset/archive.zip  


In [ ]:
!unzip /content/BTP/dataset/archive.zip

Archive:  /content/BTP/dataset/archive.zip
  inflating: drawings/spiral/testing/healthy/V01HE01.png  
  inflating: drawings/spiral/testing/healthy/V02HE01.png  
  inflating: drawings/spiral/testing/healthy/V03HE1.png  
  inflating: drawings/spiral/testing/healthy/V04HE01.png  
  inflating: drawings/spiral/testing/healthy/V05HE01.png  
  inflating: drawings/spiral/testing/healthy/V06HE01.png  
  inflating: drawings/spiral/testing/healthy/V07HE01.png  
  inflating: drawings/spiral/testing/healthy/V08HE01.png  
  inflating: drawings/spiral/testing/healthy/V09HE01.png  
  inflating: drawings/spiral/testing/healthy/V10HE01.png  
  inflating: drawings/spiral/testing/healthy/V11HE01.png  
  inflating: drawings/spiral/testing/healthy/V55HE12.png  
  inflating: drawings/spiral/testing/healthy/V55HE13.png  
  inflating: drawings/spiral/testing/healthy/V55HE14.png  
  inflating: drawings/spiral/testing/healthy/V55HE15.png  
  inflating: drawings/spiral/testing/parkinson/V01PE01.png  
  inflating:

In [ ]:
#TRAIN-TEST PATH
Spiral_Train_Path = Path("/content/spiral/training")
Spiral_Test_Path = Path("/content/spiral/testing")

In [ ]:
#making in list
Spiral_Train_PNG_Path = list(Spiral_Train_Path.glob(r"*/*.png"))
Spiral_Test_PNG_Path = list(Spiral_Test_Path.glob(r"*/*.png"))

In [ ]:
#labels
Spiral_Train_PNG_Labels = list(map(lambda x: os.path.split(os.path.split(x)[0])[1],Spiral_Train_PNG_Path))
Spiral_Test_PNG_Labels = list(map(lambda x: os.path.split(os.path.split(x)[0])[1],Spiral_Test_PNG_Path))

In [ ]:
#TRANSFORMATION TO SERIES STRUCTURE
#spiral
Spiral_Train_PNG_Path_Series = pd.Series(Spiral_Train_PNG_Path,name="PNG").astype(str)
Spiral_Train_PNG_Labels_Series = pd.Series(Spiral_Train_PNG_Labels,name="CATEGORY")

In [ ]:
Spiral_Test_PNG_Path_Series = pd.Series(Spiral_Test_PNG_Path,name="PNG").astype(str)
Spiral_Test_PNG_Labels_Series = pd.Series(Spiral_Test_PNG_Labels,name="CATEGORY")

In [ ]:
# TRANSFORMATION TO DATAFRAME STRUCTURE
#spiral
Main_Spiral_Train_Data = pd.concat([Spiral_Train_PNG_Path_Series,Spiral_Train_PNG_Labels_Series],axis=1)

In [ ]:
Main_Spiral_Train_Data

,PNG,CATEGORY
0,/content/spiral/training/healthy/V02HE02.png,healthy
1,/content/spiral/training/healthy/V55HE06.png,healthy
2,/content/spiral/training/healthy/V05HE03.png,healthy
3,/content/spiral/training/healthy/V06HE02.png,healthy
4,/content/spiral/training/healthy/V55HE09.png,healthy
...,...,...
67,/content/spiral/training/parkinson/V03PE03.png,parkinson
68,/content/spiral/training/parkinson/V03PE09.png,parkinson
69,/content/spiral/training/parkinson/V12PE02.png,parkinson
70,/content/spiral/training/parkinson/V15PE02.png,parkinson


In [ ]:
print(Main_Spiral_Train_Data.head(-1))

                                               PNG   CATEGORY
0     /content/spiral/training/healthy/V02HE02.png    healthy
1     /content/spiral/training/healthy/V55HE06.png    healthy
2     /content/spiral/training/healthy/V05HE03.png    healthy
3     /content/spiral/training/healthy/V06HE02.png    healthy
4     /content/spiral/training/healthy/V55HE09.png    healthy
..                                             ...        ...
66  /content/spiral/training/parkinson/V07PE03.png  parkinson
67  /content/spiral/training/parkinson/V03PE03.png  parkinson
68  /content/spiral/training/parkinson/V03PE09.png  parkinson
69  /content/spiral/training/parkinson/V12PE02.png  parkinson
70  /content/spiral/training/parkinson/V15PE02.png  parkinson

[71 rows x 2 columns]


In [ ]:
Main_Spiral_Test_Data = pd.concat([Spiral_Test_PNG_Path_Series,Spiral_Test_PNG_Labels_Series],axis=1)

In [ ]:
print(Main_Spiral_Test_Data.head(-1))

                                              PNG   CATEGORY
0     /content/spiral/testing/healthy/V02HE01.png    healthy
1     /content/spiral/testing/healthy/V55HE14.png    healthy
2     /content/spiral/testing/healthy/V04HE01.png    healthy
3     /content/spiral/testing/healthy/V07HE01.png    healthy
4     /content/spiral/testing/healthy/V08HE01.png    healthy
5     /content/spiral/testing/healthy/V55HE13.png    healthy
6     /content/spiral/testing/healthy/V05HE01.png    healthy
7     /content/spiral/testing/healthy/V55HE12.png    healthy
8     /content/spiral/testing/healthy/V06HE01.png    healthy
9     /content/spiral/testing/healthy/V55HE15.png    healthy
10    /content/spiral/testing/healthy/V10HE01.png    healthy
11    /content/spiral/testing/healthy/V09HE01.png    healthy
12    /content/spiral/testing/healthy/V11HE01.png    healthy
13    /content/spiral/testing/healthy/V01HE01.png    healthy
14     /content/spiral/testing/healthy/V03HE1.png    healthy
15  /content/spiral/test

In [ ]:
#SHUFFLING
Main_Spiral_Train_Data = Main_Spiral_Train_Data.sample(frac=1).reset_index(drop=True)
Main_Spiral_Test_Data = Main_Spiral_Test_Data.sample(frac=1).reset_index(drop=True)

In [ ]:
#for training
Spiral_New_JPG_Path = []
for i in range(0,72):
    x_train = cv2.imread(Main_Spiral_Train_Data["PNG"][i])
    x_train = np.array(x_train).astype("float32")
    x_train = x_train.mean()
    Spiral_New_JPG_Path.append(x_train)
Spiral_New_JPG_Path_Series = pd.Series(Spiral_New_JPG_Path,name="PNG")
encode = LabelEncoder()
Spiral_New_JPG_Labels = encode.fit_transform(Main_Spiral_Train_Data["CATEGORY"])
Spiral_New_JPG_Labels_Series = pd.Series(Spiral_New_JPG_Labels,name="CATEGORY")
Main_Spiral_New_Data = pd.concat([Spiral_New_JPG_Path_Series,Spiral_New_JPG_Labels_Series],axis=1) #This is a sequence or mapping of Series, DataFrame, or Panel objects.
#TRAIN
x_train = Main_Spiral_New_Data[["PNG"]]
y_train = Main_Spiral_New_Data["CATEGORY"]

In [ ]:
#print(x_train.shape, y_train.shape)

In [ ]:
##MODELS
#from sklearn import svm
#import numpy as np
#from sklearn.metrics import f1_score
#c=svm.SVC(kernel = 'linear',gamma = 'scale', shrinking = False,)
#c.fit(x_train,y_train)
#predict=c.predict(x_train)

In [ ]:
#R2CV = cross_val_score(c,x_train,np.array(y_train),verbose=False).mean()
#error = -cross_val_score(c,x_train,y_train,scoring="neg_mean_squared_error",verbose=False).mean()
#print("ACCURACY-->",accuracy_score(y_train,predict))
#print("R2CV-->",R2CV*100) #cross validations
#print("MEAN SQUARED ERROR-->",np.sqrt(error))#mse
#print("F1 score-->",f1_score(y_train,predict))
#print("-" * 30)

In [ ]:
#for testing
Spiral_New_JPG_Path = []
for i in range(0,30):
    x_test = cv2.imread(Main_Spiral_Test_Data["PNG"][i])
    x_test = np.array(x_test).astype("float32")
    x_test = x_test.mean()
    Spiral_New_JPG_Path.append(x_test)
Spiral_New_JPG_Path_Series = pd.Series(Spiral_New_JPG_Path,name="PNG")
encode = LabelEncoder()
Spiral_New_JPG_Labels = encode.fit_transform(Main_Spiral_Test_Data["CATEGORY"])
Spiral_New_JPG_Labels_Series = pd.Series(Spiral_New_JPG_Labels,name="CATEGORY")
Main_Spiral_New_Data = pd.concat([Spiral_New_JPG_Path_Series,Spiral_New_JPG_Labels_Series],axis=1)
#TEST
x_test = Main_Spiral_New_Data[["PNG"]]
y_test = Main_Spiral_New_Data["CATEGORY"]

In [ ]:
#print(x_test.shape, y_test.shape)

In [ ]:
#MODELS
#from sklearn import svm
#import numpy as np
#from sklearn.metrics import f1_score
#c=svm.SVC(kernel = 'linear',gamma = 'scale', shrinking = False,)
#c.fit(x_test,y_test)
#predict=c.predict(x_test)

In [ ]:
#R2CV = cross_val_score(c,x_test,np.array(y_test),verbose=False).mean()
#error = -cross_val_score(c,x_test,y_test,scoring="neg_mean_squared_error",verbose=False).mean()
#print("ACCURACY-->",accuracy_score(y_test,predict))
#print("R2CV-->",R2CV*100) #cross validations
##print("MEAN SQUARED ERROR-->",np.sqrt(error))#mse
#print("F1 score-->",f1_score(y_test,predict))
#print("-" * 30)

In [ ]:
print(x_train.shape, y_train.shape, x_test.shape, y_test.shape)

(72, 1) (72,) (30, 1) (30,)


In [ ]:
#MODELS
from sklearn import svm
import numpy as np
from sklearn.metrics import f1_score
c=svm.SVC(kernel = 'linear',gamma = 'scale', shrinking = False,)
c.fit(x_train,y_train)
predict=c.predict(x_test)

In [ ]:
R2CV = cross_val_score(c,x_test,np.array(y_test),verbose=False).mean()
error = -cross_val_score(c,x_test,y_test,scoring="neg_mean_squared_error",verbose=False).mean()
print("ACCURACY-->",accuracy_score(y_test,predict))
print("R2CV-->",R2CV*100) #cross validations
print("MEAN SQUARED ERROR-->",np.sqrt(error))#mse
print("F1 score-->",f1_score(y_test,predict))
print("-" * 30)

ACCURACY--> 0.5666666666666667
R2CV--> 56.666666666666664
MEAN SQUARED ERROR--> 0.6582805886043833
F1 score--> 0.5517241379310344
------------------------------
